# OpenFold Local Notebook

Provides the flexibility to run inference using a local installation of OpenFold with Docker, along with the convenience of visualizing results using the same plots from the OpenFold Colab Notebook.

If you have access to a machine and want to do quick inference and visualize results, there are some useful things you can do with this notebook:

- You can use precomputed alignments, which enables you to run inference with different model parameters to compare results.
- Get the best model and plots using the provided utility functions.
- Handle long-running executions.


Of course, you can do this solely using Docker commands in the terminal, but you would need to code/adjust the Colab functions to work with data locally. This notebook gives you a head start.

## Setup the notebook

Start your notebook:

Go to the notebook folder:

`cd notebooks`

Install the requirements in your env:

`pip install -r utils/requirements.txt`

Start your Jupyter server

`jupyter lab . --ip="0.0.0.0"`



## Running Inference 

### Example usage 1: Using a sequence string

In [ ]:
from utils.docker_commands import run_inference

# For multiple sequences, separate sequences with a colon `:`
inference_input = "DAGAQGAAIGSPGVLSGNVVQVPVHVPVNVCGNTVSVIGLLNPAFGNTCVNA:AGETGRTGVLVTSSATNDGDSGWGRFAG"

database_dir = "<change_me>/path"
model_name = "multimer"
weight_set = 'AlphaFold'

run_inference(inference_input, database_dir, weight_set, model_name)


### Example usage 2: Using a pre-existing fasta directory

In [ ]:
from utils.docker_commands import run_inference

# Example usage, put your .fasta file with the sequences to process in the inference dir
inference_dir = "<change_me>/path"
databases_dir = "<change_me>/path"


weight_set = 'AlphaFold'
model_name = "multimer"

run_inference(inference_dir, databases_dir, weight_set, model_name, use_precomputed_alignments=True)